In [1]:
import pandas as pd

# Load the datasets
applications_df = pd.read_csv('cleaned_applications.csv')
branches_df = pd.read_csv('cleaned_branches.csv')
customers_df = pd.read_csv('cleaned_customers.csv')
defaults_df = pd.read_csv('cleaned_defaults.csv')
loans_df = pd.read_csv('cleaned_loans.csv')
transactions_df = pd.read_csv('cleaned_transactions.csv')

print("Datasets loaded successfully.")

Datasets loaded successfully.


In [2]:
loan_default_merge_df = pd.merge(loans_df, defaults_df, on='Loan_ID', how='left')

default_cols = ['Default_ID', 'Default_Date', 'Default_Amount', 'Default_Reason', 'Recovery_Status', 'Recovery_Amount', 'Legal_Action']
for col in default_cols:
    if loan_default_merge_df[col].dtype == 'float64' or loan_default_merge_df[col].dtype == 'int64':
        loan_default_merge_df[col] = loan_default_merge_df[col].fillna(0)
    elif loan_default_merge_df[col].dtype == 'object':
        loan_default_merge_df[col] = loan_default_merge_df[col].fillna('Not Applicable')

loan_default_merge_df['Default_Flag'] = loan_default_merge_df['Default_ID'].apply(lambda x: 0 if x == 'Not Applicable' else 1)

display(loan_default_merge_df.head())

Loan_ID Customer_ID_x  Loan_Amount  Interest_Rate  Loan_Term  \
0  L000001       C045929      4030468           8.45         24   
1  L000002       C038991      3498647          11.90         48   
2  L000003       C046662      4871438           9.74         24   
3  L000004       C040268       541515          13.55         24   
4  L000005       C067057       631036          10.10         36   

  Disbursal_Date Repayment_Start_Date Repayment_End_Date Loan_Status  \
0     2023-03-25           2023-04-25         2025-03-25      Active   
1     2023-10-23           2023-11-23         2027-10-23      Closed   
2     2024-07-30           2024-08-30         2026-07-30     Overdue   
3     2024-11-22           2024-12-22         2026-11-22      Active   
4     2024-03-30           2024-04-30         2027-03-30     Overdue   

   Overdue_Amount  ...  Collateral_Details      Default_ID Customer_ID_y  \
0               0  ...            Property  Not Applicable           NaN   
1           44003  ...            Property  Not Applicable           NaN   
2               0  ...      Not Applicable  Not Applicable           NaN   
3               0  ...      Not Applicable  Not Applicable           NaN   
4               0  ...      Not Applicable  Not Applicable           NaN   

     Default_Date Default_Amount  Default_Reason Recovery_Status  \
0  Not Applicable            0.0  Not Applicable  Not Applicable   
1  Not Applicable            0.0  Not Applicable  Not Applicable   
2  Not Applicable            0.0  Not Applicable  Not Applicable   
3  Not Applicable            0.0  Not Applicable  Not Applicable   
4  Not Applicable            0.0  Not Applicable  Not Applicable   

  Recovery_Amount    Legal_Action Default_Flag  
0             0.0  Not Applicable            0  
1             0.0  Not Applicable            0  
2             0.0  Not Applicable            0  
3             0.0  Not Applicable            0  
4             0.0  Not Applicable            0  

[5 rows x 21 columns]

In [3]:
# Define EMI bins using quantiles
emi_bins = loan_default_merge_df['EMI_Amount'].quantile([0, 0.25, 0.5, 0.75, 1]).tolist()

# Create a new column for EMI bins
loan_default_merge_df['EMI_Bin'] = pd.cut(loan_default_merge_df['EMI_Amount'], bins=emi_bins, include_lowest=True, labels=[f'Q{i+1}' for i in range(len(emi_bins)-1)])

# Group by EMI bins and calculate default rate
emi_default_rates = loan_default_merge_df.groupby('EMI_Bin')['Default_Flag'].mean().reset_index()

# Rename columns for clarity
emi_default_rates.rename(columns={'Default_Flag': 'Default_Rate'}, inplace=True)

# Display the results
display(emi_default_rates)

/var/folders/7v/b4cx31gj2m15lfjrqk3sy1240000gp/T/ipykernel_10322/609143533.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  emi_default_rates = loan_default_merge_df.groupby('EMI_Bin')['Default_Flag'].mean().reset_index()


EMI_Bin  Default_Rate
0      Q1      0.098426
1      Q2      0.098868
2      Q3      0.100995
3      Q4      0.099673

In [4]:
print("EMI Default Rates:")
display(emi_default_rates)

# Analyze the default rates across bins
min_default_rate = emi_default_rates['Default_Rate'].min()
max_default_rate = emi_default_rates['Default_Rate'].max()
range_default_rate = max_default_rate - min_default_rate

print(f"\nMinimum Default Rate: {min_default_rate:.4f}")
print(f"Maximum Default Rate: {max_default_rate:.4f}")
print(f"Range of Default Rates: {range_default_rate:.4f}")

# Check for significant increases
# A simple check: is the max rate significantly higher than the min rate?
# A threshold could be considered if a bin's default rate is much higher than the previous one.
# Given the small range, a significant increase is unlikely based on these bins.
print("\nAnalysis of Default Rates Across EMI Bins:")
if range_default_rate < 0.01: # Define a threshold for "significant" difference, e.g., less than 1%
    print("The range of default rates across EMI bins is small. There are no obvious EMI thresholds based on these quartiles that indicate significantly higher default risk.")
else:
    print("There might be some variation in default rates across EMI bins. Further investigation is needed to identify specific thresholds.")

# Based on the analysis of the emi_default_rates, conclude whether there are any obvious EMI thresholds based on these quartiles that indicate significantly higher default risk.
print("\nConclusion:")
if range_default_rate < 0.01:
    print("Based on the quartile analysis, there are no obvious EMI thresholds that indicate significantly higher default risk.")
else:
    print("Based on the quartile analysis, there might be some EMI thresholds that indicate significantly higher default risk. Further analysis with finer bins or other methods is recommended.")

EMI Default Rates:


EMI_Bin  Default_Rate
0      Q1      0.098426
1      Q2      0.098868
2      Q3      0.100995
3      Q4      0.099673


Minimum Default Rate: 0.0984
Maximum Default Rate: 0.1010
Range of Default Rates: 0.0026

Analysis of Default Rates Across EMI Bins:
The range of default rates across EMI bins is small. There are no obvious EMI thresholds based on these quartiles that indicate significantly higher default risk.

Conclusion:
Based on the quartile analysis, there are no obvious EMI thresholds that indicate significantly higher default risk.


In [5]:
# Merge loans_df with applications_df to get 'Loan_Purpose'
loans_applications_df = pd.merge(loans_df, applications_df[['Loan_ID', 'Loan_Purpose']], on='Loan_ID', how='left')

# Group by 'Loan_Purpose' and calculate the mean EMI amount
emi_by_loan_type = loans_applications_df.groupby('Loan_Purpose')['EMI_Amount'].mean().reset_index()

# Display the results
display(emi_by_loan_type)

Loan_Purpose    EMI_Amount
0         Business  51138.559771
1        Education  50997.858482
2  Home Renovation  50986.143347
3         Personal  50437.003658
4          Vehicle  50715.325768

In [6]:
print("Summary of EMI Analysis and Default Risk:")
print("\nRelationship between EMI Amounts and Default Probabilities:")
print(f"Based on the quartile analysis of EMI amounts, the default rates are relatively consistent across all bins, ranging from {min_default_rate:.2%} to {max_default_rate:.2%}.")
print("The small range of default rates (approximately {:.2%}) suggests that within these broad quartiles, there are no clear thresholds for EMI amounts that indicate a significantly higher default risk.".format(range_default_rate))

print("\nComparison of Average EMI Amounts Across Loan Types:")
display(emi_by_loan_type)
highest_emi_type = emi_by_loan_type.loc[emi_by_loan_type['EMI_Amount'].idxmax()]
lowest_emi_type = emi_by_loan_type.loc[emi_by_loan_type['EMI_Amount'].idxmin()]
print(f"Analysis of average EMI amounts by loan type shows variations.")
print(f"The loan type with the highest average EMI is '{highest_emi_type['Loan_Purpose']}' at {highest_emi_type['EMI_Amount']:.2f}.")
print(f"The loan type with the lowest average EMI is '{lowest_emi_type['Loan_Purpose']}' at {lowest_emi_type['EMI_Amount']:.2f}.")

print("\nOverall Overview:")
print("The analysis indicates that while average EMI amounts vary across different loan types, the default probability does not show a strong correlation with EMI amount when examined in broad quartiles. This suggests that other factors may have a more significant impact on default risk than the EMI amount alone within the ranges observed in this dataset.")

Summary of EMI Analysis and Default Risk:

Relationship between EMI Amounts and Default Probabilities:
Based on the quartile analysis of EMI amounts, the default rates are relatively consistent across all bins, ranging from 9.84% to 10.10%.
The small range of default rates (approximately 0.26%) suggests that within these broad quartiles, there are no clear thresholds for EMI amounts that indicate a significantly higher default risk.

Comparison of Average EMI Amounts Across Loan Types:


Loan_Purpose    EMI_Amount
0         Business  51138.559771
1        Education  50997.858482
2  Home Renovation  50986.143347
3         Personal  50437.003658
4          Vehicle  50715.325768

Analysis of average EMI amounts by loan type shows variations.
The loan type with the highest average EMI is 'Business' at 51138.56.
The loan type with the lowest average EMI is 'Personal' at 50437.00.

Overall Overview:
The analysis indicates that while average EMI amounts vary across different loan types, the default probability does not show a strong correlation with EMI amount when examined in broad quartiles. This suggests that other factors may have a more significant impact on default risk than the EMI amount alone within the ranges observed in this dataset.
